# **Sentimen Analisis Review APK Play Store**



# **Import Library**

In [11]:
!pip install google-play-scraper

In [12]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

In [8]:
import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

In [4]:
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stop words) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Scapping Dataset


In [26]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

# Mengambil semua ulasan dari aplikasi dengan ID 'com.mobile.legend' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.mobile.legends',  # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count= 17000,           # Jumlah maksimum ulasan yang ingin diambil
    filter_score_with=None # Mengambil ulasan dengan rating
)

In [27]:
# Menyimpan ulasan dalam file CSV
import csv

with open('mobile_legend.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Rating', 'Review', ])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['score'], review['content']])  # Menulis konten ulasan ke dalam file CSV

# Loading Dataset

In [28]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('mobile_legend.csv', index=False)

In [29]:
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape


In [30]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9bd15e3b-7ae1-4827-af1c-f80b9aea4fa2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tiap main rank pasti aja ada player beban/male...,1,18353,1.9.65.10602,2025-03-28 15:42:15,None,NaT,1.9.65.10602
1,04893854-ffc4-4a5a-bd53-18dda4879354,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,jika setiap update game rank nya selalu saja m...,2,606,1.9.65.10602,2025-03-30 16:27:41,None,NaT,1.9.65.10602
2,32d316bf-a701-4017-9746-8c3b71dc0892,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bintang 3 aja moonton, grafik sudah oke, efek ...",3,1478,1.9.65.10602,2025-03-29 18:36:17,"Halo Kak,\nKami berkomitmen untuk menciptakan ...",2025-01-04 04:00:00,1.9.65.10602
3,b541e597-ea22-4560-a4c2-a46643e719e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kemarin install lagi ini game dengan harapan s...,1,613,1.9.65.10602,2025-03-28 23:50:06,None,NaT,1.9.65.10602
4,45f19d76-0bd1-4469-8cc8-ddfb1ecc7896,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"untuk gamenya sudah bagus, banyak skin dan fit...",3,42,1.9.65.10602,2025-04-10 06:11:59,None,NaT,1.9.65.10602


In [31]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207000 entries, 0 to 206999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              207000 non-null  object        
 1   userName              207000 non-null  object        
 2   userImage             207000 non-null  object        
 3   content               207000 non-null  object        
 4   score                 207000 non-null  int64         
 5   thumbsUpCount         207000 non-null  int64         
 6   reviewCreatedVersion  165154 non-null  object        
 7   at                    207000 non-null  datetime64[ns]
 8   replyContent          7788 non-null    object        
 9   repliedAt             7788 non-null    datetime64[ns]
 10  appVersion            165154 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 17.4+ MB


In [32]:
# Hanya ambil data yang kolom 'content' dan 'score'-nya tidak kosong
clean_df = app_reviews_df[app_reviews_df['content'].notna() & app_reviews_df['score'].notna()]

# Reset index agar rapi
clean_df = clean_df.reset_index(drop=True)

# Cek hasilnya
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207000 entries, 0 to 206999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              207000 non-null  object        
 1   userName              207000 non-null  object        
 2   userImage             207000 non-null  object        
 3   content               207000 non-null  object        
 4   score                 207000 non-null  int64         
 5   thumbsUpCount         207000 non-null  int64         
 6   reviewCreatedVersion  165154 non-null  object        
 7   at                    207000 non-null  datetime64[ns]
 8   replyContent          7788 non-null    object        
 9   repliedAt             7788 non-null    datetime64[ns]
 10  appVersion            165154 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 17.4+ MB


In [33]:
# Salin DataFrame utama (opsional untuk keamanan)
df_clean = app_reviews_df.copy()

# DROP kolom dengan mayoritas missing value
df_clean = df_clean.drop(columns=['replyContent', 'repliedAt'])

# DROP baris yang missing pada kolom penting jika ingin memakai reviewCreatedVersion dan appVersion
df_clean = df_clean.dropna(subset=['reviewCreatedVersion', 'appVersion'])

# Cek kembali info setelah pembersihan
df_clean.info()


# Tampilkan jumlah data akhir
print(f"\nJumlah data setelah penanganan missing value: {len(df_clean)}")


<class 'pandas.core.frame.DataFrame'>
Index: 165154 entries, 0 to 206999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              165154 non-null  object        
 1   userName              165154 non-null  object        
 2   userImage             165154 non-null  object        
 3   content               165154 non-null  object        
 4   score                 165154 non-null  int64         
 5   thumbsUpCount         165154 non-null  int64         
 6   reviewCreatedVersion  165154 non-null  object        
 7   at                    165154 non-null  datetime64[ns]
 8   appVersion            165154 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 12.6+ MB

Jumlah data setelah penanganan missing value: 165154


In [34]:
clean_df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,41846
at,0
replyContent,199212
repliedAt,199212


In [35]:
# Drop kolom yang banyak missing dan tidak digunakan
clean_df.drop(columns=["replyContent", "repliedAt", "reviewCreatedVersion", "appVersion"], inplace=True)


In [ ]:
# # Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
# clean_df = app_reviews_df.dropna()

In [36]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

##Preprocessing

In [16]:
slangwords = {
    'lancar': 'stabil', 'laporkan': 'melaporkan', 'id': 'identitas', 'kondisi': 'situasi',
    'terhitung': 'tercatat', 'update': 'pembaruan', 'layar': 'tampilan', 'awal': 'awal',
    'terputus': 'putus', 'padahal': 'meskipun', 'punishment': 'hukuman', 'jelas': 'pasti',
    'dibiarin': 'dibiarkan', 'giliran': 'saat', 'yg': 'yang',
    'sungguh': 'serius', 'dihilangkan': 'dihapuskan', 'dikurangi': 'dikurangkan', 'ajukan': 'ajukan',
    'bisa': 'mampu', 'makin': 'semakin', 'diloby': 'di lobi', 'dipertarungkan': 'dipertandingkan',
    'dark': 'buruk', 'minus': 'merugikan', 'tolol': 'bodoh', 'season': 'musim', 'perbaiki': 'perbaiki',
    'war': 'pertempuran', 'monton': 'moonton', 'sinyal': 'jaringan', 'ijo': 'hijau',
    'baguss': 'bagus', 'terimakasih': 'terima kasih', 'gameplay': 'permainan', 'mudarat': 'kerugian',
    'capek': 'lelah', 'mulu': 'terus-menerus', 'kek': 'seperti', 'ff': 'free fire', 'ngabisin': 'menghabiskan',
    'keseruannya': 'keseruan', 'sia': 'sia-sia', 'tidak merekomendasikan': 'tidak menyarankan', 'ban': 'blokir',
    'pencocokan': 'penyesuaian', 'musuh': 'lawan', 'starlight': 'pemain elit', 'beda': 'berbeda',
    'tinggi': 'tinggi', 'tim': 'kelompok', 'perpanjang': 'perpanjangan', 'tombol': 'tombol',
    'naik': 'meningkat', 'rugi': 'merugi', 'push': 'naik peringkat', 'bintang': 'nilai',
    'selanjutnya': 'berikutnya', 'dst': 'dan seterusnya', 'download': 'unduh', 'hentikan': 'berhenti',
    'nyampe': 'sampai', 'perbaikilah': 'perbaiki', 'meresahkan': 'mengganggu', 'pensi': 'berhenti',
    'lane': 'jalur', 'gimana': 'bagaimana', 'ganggu': 'mengganggu', 'bagus': 'baik', 'skor': 'nilai',
    'diawal': 'di awal', 'wifi': 'jaringan', 'mati': 'terputus', 'login': 'masuk',
    'ditingkatkan': 'diperbaiki', 'ngebug': 'bug', 'lemot': 'lambat', 'ngestuck': 'macet',
    'ngecrash': 'crash', 'ngaco': 'berantakan', 'ngilangin': 'menghilangkan', 'ngasih': 'memberikan',
    'jelekk': 'buruk', 'jelek': 'buruk', 'nge-lag': 'lag', 'ngikut': 'mengikuti',
    'kebanyakan': 'terlalu banyak', 'kesel': 'kesal', 'sampe': 'sampai', 'sampek': 'sampai',
    'bikin': 'membuat', 'maksa': 'memaksa', 'mending': 'lebih baik', 'gitu': 'seperti itu',
    'kayak': 'seperti', 'bangett': 'banget', 'pdhl': 'padahal', 'udh': 'sudah', 'udah': 'sudah',
    'btw': 'ngomong-ngomong', 'ni': 'ini', 'dahl': 'padahal', 'dr': 'dari', 'klo': 'kalau',
    'kalo': 'kalau', 'lg': 'lagi', 'jg': 'juga', 'dpt': 'dapat', 'tp': 'tapi', 'tmn': 'teman',
    'doang': 'saja', 'bgtu': 'begitu', 'cmn': 'cuma', 'cuma': 'hanya', 'bkin': 'membuat',
    'ml': 'mobile legends', 'skill': 'kemampuan', 'emblem': 'atribut', 'win': 'menang',
    'winstreak': 'menang beruntun', 'fps': 'frame per second', 'cs': 'layanan pelanggan',
    'mvp': 'pemain terbaik', 'crash': 'berhenti mendadak', 'mohon': 'harap', 'player': 'pemain',
    'players': 'pemain', 'egois': 'mementingkan diri sendiri', 'troll': 'pengganggu',
    'matching': 'penjodohan', 'matchingnya': 'penjodohan pertandingan', 'dark system': 'sistem buruk',
    'game sebelah': 'permainan kompetitor', 'delay': 'penundaan', 'team war': 'pertempuran tim',
    'ws': 'menang beruntun', 'tier': 'tingkatan', 'legend': 'tingkatan legend', 'depressi': 'stres',
    'depresi': 'stres', 'buff': 'penambah kekuatan', 'kill': 'mengalahkan', 'toxic': 'kasar',
    'role': 'peran','gaubah': 'tidak diubah', 'mainnya': 'permainannya', 'hp': 'ponsel', 'masuk2': 'masuk-masuk',
    'dimaksimalkan': 'dioptimalkan', 'pesan': 'notifikasi', 'awlnya': 'awalnya', 'gajelas': 'tidak jelas',
    'ngefreeze': 'beku', 'ngechat': 'mengirim pesan', 'ngeleg': 'lag', 'stuck': 'macet',
    'gw': 'saya', 'gua': 'saya', 'gue': 'saya', 'gk': 'tidak', 'ga': 'tidak', 'gak': 'tidak', 'g': 'tidak',
    'tdk': 'tidak', 'trs': 'terus', 'gbs': 'tidak bisa', 'aj': 'saja', 'aja': 'saja', 'bgt': 'banget',
    'bgd': 'banget', 'bngt': 'banget', 'bngt2': 'banget-banget', 'bae': 'baik', 'parahh': 'parah',
    'parahhh': 'parah', 'dm': 'diamond', 'match': 'pertandingan', 'relag': 'relog', 'relog': 'masuk ulang',
    'nggak': 'tidak', 'ngak': 'tidak', 'ngga': 'tidak', 'pvp': 'pertarungan pemain',
    'ms': 'milidetik', 'afk': 'tidak aktif', 'op': 'overpowered', 'chat': 'pesan', 'heronya': 'hero-nya',
    'dev': 'pengembang', 'eventnya': 'acara-nya', 'event': 'acara', 'buq': 'bug', 'bugnya': 'bug-nya',
    'spin': 'putaran', 'ranked': 'peringkat', 'matchmaking': 'pencocokan pertandingan',
    'battle': 'pertempuran', 'skin': 'kulit karakter', 'loby': 'lobi', 'loading': 'memuat',
    'scam': 'penipuan', 'cheat': 'curang', 'credit score': 'skor kredit', 'score': 'skor',
    'resource': 'sumber daya', 'ping': 'latensi', 'eror': 'error', 'koneksi': 'jaringan',
    'ngelag': 'koneksi lambat', 'nge lag': 'koneksi lambat', 'ngestuck': 'macet', 'afk': 'tidak aktif', 'troll': 'bermain merugikan tim',
    'ngetroll': 'bermain merugikan tim', 'dark system': 'sistem tidak adil', 'drak sistem': 'sistem tidak adil', 'beban': 'pemain yang membebani tim',
    'matchmaking': 'pencocokan pertandingan', 'match making': 'pencocokan pertandingan', 'credit score': 'skor kelakuan', 'mvp': 'pemain terbaik',
    'auto lose': 'pasti kalah', 'winrate': 'persentase kemenangan', 'bug': 'kesalahan sistem', 'bugnya': 'kesalahan sistem',
    'ulti': 'jurus pamungkas', 'dapet tim tolol': 'mendapat tim buruk', 'tim tolol': 'tim buruk', 'musuh jago': 'lawan hebat', 'server': 'peladen',
    'serfer': 'peladen', 'monton': 'moonton', 'moonton': 'pengembang', 'speed mode': 'mode kecepatan', 'banting hp': 'melempar hp', 'ml': 'mobile legends',
    'mobile lagend': 'mobile legends','mobile': 'mobile legends','auto lose': 'pasti kalah', 'winrate': 'persentase kemenangan','bugnya': 'kesalahan sistem',
    'ulti': 'jurus pamungkas', 'dapet tim tolol': 'mendapat tim buruk', 'tim tolol': 'tim buruk', 'musuh jago': 'lawan hebat', 'server': 'peladen',
    'serfer': 'peladen', 'monton': 'moonton', 'moonton': 'pengembang', 'speed mode': 'mode kecepatan', 'banting hp': 'melempar hp', 'ml': 'mobile legends',
    'mobile lagend': 'mobile legends', 'mobile lagand': 'mobile legends', 'kntl': 'penis', 'kontol': 'penis', 'anj': 'anjing', 'bangkek': 'bangkai',
    'ancvkkk': 'kata kasar', 'pensi': 'pensiun', 'pensiun aja': 'berhenti bermain', 'gblok': 'bodoh', 'tolol': 'bodoh', 'bego': 'bodoh', 'tim kayak bot': 'tim sangat buruk',
    'bot': 'kecerdasan buatan', 'cheat': 'curang', 'custom mode': 'mode khusus', 'push rank': 'naik peringkat', 'joki': 'pemain bayaran', 'pick': 'memilih hero',
    'draft pick': 'pemilihan hero', 'op': 'terlalu kuat', 'stun': 'tidak bisa bergerak', 'login': 'masuk', 'resource': 'sumber daya',
    'loading': 'memuat', 'lag': 'koneksi lambat', 'dc': 'terputus', 'lemot': 'lambat', 'hero': 'karakter', 'skin': 'kulit karakter', 'event': 'acara',
    'eventnya': 'acara-nya', 'spin': 'putaran', 'ranked': 'peringkat', 'battle': 'pertempuran', 'loby': 'lobi', 'scam': 'penipuan', 'score': 'skor', 'ping': 'latensi',
    'eror': 'error', 'dev': 'pengembang', 'koneksi': 'jaringan','Gak' : 'tidak', 'garena': 'karena', 'nih': 'ini','Min' : 'admin', 'y': 'iya', 'ngadepin': 'menghadapi',
    'nie':'ini','sma':'sama','mmr':'memori','kayak':'sangat','emang':'memang','trims': 'terima kasih','mksh': 'terima kasih','nih': 'ini','mlbb' : 'mobile legend bang bang','perbaikiterkadang' : 'perbaiki terkadang',
    'ajabisa': 'aja bisa', 'dapet' : 'dapat', 'sumpahh': 'sumpah', 'bebanmales':'beban malas'

}

import re
def replace_slang(text, slang_dict):
    # Tokenisasi kata dan tetap mempertahankan tanda baca
    words = re.findall(r'\b\w+\b|[^\w\s]', text.lower())
    cleaned_words = []

    for word in words:
        # Cek dan ganti dengan kata baku jika ada di kamus
        cleaned_word = slang_dict.get(word, word)
        cleaned_words.append(cleaned_word)

    # Gabungkan kembali jadi kalimat
    return ' '.join(cleaned_words)


In [21]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy","lo","aja"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [22]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,at
0,9bd15e3b-7ae1-4827-af1c-f80b9aea4fa2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tiap main rank pasti aja ada player beban/male...,1,18237,2025-03-28 15:42:15
1,04893854-ffc4-4a5a-bd53-18dda4879354,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,jika setiap update game rank nya selalu saja m...,2,595,2025-03-30 16:27:41
2,32d316bf-a701-4017-9746-8c3b71dc0892,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Bintang 3 aja moonton, grafik sudah oke, efek ...",3,1470,2025-03-29 18:36:17
3,b541e597-ea22-4560-a4c2-a46643e719e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kemarin install lagi ini game dengan harapan s...,1,603,2025-03-28 23:50:06
4,45f19d76-0bd1-4469-8cc8-ddfb1ecc7896,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"untuk gamenya sudah bagus, banyak skin dan fit...",3,39,2025-04-10 06:11:59
...,...,...,...,...,...,...,...
206995,12a56712-5518-49cc-8051-741e56f11150,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Moonton is the best game developer, uda ls 7x ...",1,0,2024-03-09 17:26:29
206996,5d5bf593-9934-4945-a5c7-a5492bc2d782,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dear moonton. Tolong pisahkan games mc. Dan su...,5,0,2020-10-19 09:00:59
206997,6906b977-8711-4b3d-9a67-ff53f0659c25,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Game nya bagus tapi kok sesudah update kok gak...,1,0,2023-02-14 05:43:05
206998,5daf18d9-1619-48e4-99c1-4ee3588a8779,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolongnya sinyal bagus 4g tiap d tengah2 merah...,1,0,2022-09-10 23:01:19


In [23]:
import nltk
from nltk.corpus import stopwords # Import stopwords from nltk.corpus

# nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource


# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(lambda x: replace_slang(x, slangwords))

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Define stop_words here using the imported stopwords
stop_words = set(stopwords.words('indonesian')) # Change 'english' to 'indonesian' for Indonesian stopwords

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

# Tampilkan hasil preprocessing
print(clean_df[['content', 'text_akhir']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


                                             content  \
0  tiap main rank pasti aja ada player beban/male...   
1  jika setiap update game rank nya selalu saja m...   
2  Bintang 3 aja moonton, grafik sudah oke, efek ...   
3  kemarin install lagi ini game dengan harapan s...   
4  untuk gamenya sudah bagus, banyak skin dan fit...   

                                          text_akhir  
0  main rank pemain beban malas main pemain penga...  
1  pembaruan game rank mengurang lelah menyelesai...  
2  nilai pengembang grafik oke efek visual kemamp...  
3  kemarin install game harapan semoga kendala ra...  
4  gamenya kulit karakter fitur fitur menarik gam...  


In [ ]:
!pip install scikit-learn

##Pelabelan

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Fungsi untuk menentukan sentiment berdasarkan skor
def get_sentiment(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'  # Ini masih ada, tapi nanti akan dihapus
    else:
        return 'negative'

# Menambahkan kolom 'sentiment' berdasarkan skor
clean_df['sentiment'] = clean_df['score'].apply(get_sentiment)

# Menghapus data dengan kelas 'neutral'
filtered_df = clean_df[clean_df['sentiment'] != 'neutral']

# Menyiapkan fitur (X) dan label (y) untuk pelatihan
X = filtered_df['text_akhir']  # Menggunakan teks yang sudah dibersihkan
y = filtered_df['sentiment']  # Kolom sentiment yang hanya ada 'positive' dan 'negative'

# Mengubah teks menjadi fitur numerik menggunakan TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

# Membagi data menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Menampilkan bentuk data pelatihan dan pengujian
print(f"Bentuk data pelatihan: {X_train.shape}")
print(f"Bentuk data pengujian: {X_test.shape}")


Bentuk data pelatihan: (149487, 5000)
Bentuk data pengujian: (37372, 5000)


##Melatih model

### XGBoost

In [25]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

# Encode label ke 0 dan 1
y_train_encoded = pd.Series(y_train).map({'positive': 1, 'negative': 0})
y_test_encoded = pd.Series(y_test).map({'positive': 1, 'negative': 0})

# Hitung class weight (optional, bantu kalau data imbalance)
class_counts = Counter(y_train_encoded)
class_weights_dict = {
    0: class_counts[0] / (class_counts[0] + class_counts[1]),
    1: class_counts[1] / (class_counts[0] + class_counts[1]),
}

# Konversi ke DMatrix (format XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
dtest = xgb.DMatrix(X_test, label=y_test_encoded)

# Set hyperparameter
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 6,
    'scale_pos_weight': class_weights_dict[0] / class_weights_dict[1]
}

# Train model
num_round = 100
bst = xgb.train(params, dtrain, num_round)

# === 6. Evaluasi ===
y_pred_prob = bst.predict(dtest)
y_pred = np.round(y_pred_prob)

accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"\nAkurasi: {accuracy:.4f}")
print(classification_report(y_test_encoded, y_pred))



Akurasi: 0.7463
              precision    recall  f1-score   support

           0       0.84      0.77      0.80     24965
           1       0.60      0.71      0.65     12407

    accuracy                           0.75     37372
   macro avg       0.72      0.74      0.73     37372
weighted avg       0.76      0.75      0.75     37372



### SVM

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# y = filtered_df['sentiment']

# === 3. TF-IDF Vectorizer ===
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))
X_tfidf = tfidf_vectorizer.fit_transform(X)


# === 4. SMOTE Balancing ===
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_tfidf, y)


# === 5. Split Data ===
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, stratify=y_res, random_state=42
)

# === 6. Grid Search SVM ===
param_grid = {'C': [0.5, 1, 2]}
grid = GridSearchCV(LinearSVC(max_iter=2000), param_grid, cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

# === 7. Evaluasi ===
acc = accuracy_score(y_test, y_pred)
print(f"\nBest Parameters: {grid.best_params_}")
print(f"Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))



Best Parameters: {'C': 0.5}
Accuracy: 0.8247
              precision    recall  f1-score   support

    negative       0.82      0.83      0.83     25033
    positive       0.83      0.81      0.82     25033

    accuracy                           0.82     50066
   macro avg       0.82      0.82      0.82     50066
weighted avg       0.82      0.82      0.82     50066



### logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Logistic Regression
model = LogisticRegression(max_iter=2000)

# Melatih model
model.fit(X_train, y_train)

# Memprediksi dengan data pengujian
y_pred = model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi: {accuracy:.4f}")
print(classification_report(y_test, y_pred))


Akurasi: 0.8258
              precision    recall  f1-score   support

    negative       0.82      0.84      0.83     25033
    positive       0.83      0.81      0.82     25033

    accuracy                           0.83     50066
   macro avg       0.83      0.83      0.83     50066
weighted avg       0.83      0.83      0.83     50066



In [32]:
from sklearn.model_selection import GridSearchCV

# Definisikan grid parameter untuk Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

# GridSearchCV untuk mencari parameter terbaik
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Menampilkan hasil pencarian
print(f"Best parameters: {grid_search.best_params_}")

# Evaluasi dengan model terbaik

best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(classification_report(y_test, y_pred_best))


Best parameters: {'C': 1, 'solver': 'liblinear'}
              precision    recall  f1-score   support

    negative       0.82      0.84      0.83     25033
    positive       0.83      0.82      0.82     25033

    accuracy                           0.83     50066
   macro avg       0.83      0.83      0.83     50066
weighted avg       0.83      0.83      0.83     50066



### Naive Bayes

In [34]:
 # Model: Naive Bayes
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

Naive Bayes Model Evaluation:
Accuracy: 0.7631726121519594
              precision    recall  f1-score   support

    negative       0.74      0.82      0.78     25033
    positive       0.80      0.70      0.75     25033

    accuracy                           0.76     50066
   macro avg       0.77      0.76      0.76     50066
weighted avg       0.77      0.76      0.76     50066



### Stochastic Gradient Descent

In [35]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


# Model: Stochastic Gradient Descent
sgd_model = SGDClassifier(random_state=42)
sgd_model.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred_sgd = sgd_model.predict(X_test)
print("SGD Classifier Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))


SGD Classifier Model Evaluation:
Accuracy: 0.8193184995805537
              precision    recall  f1-score   support

    negative       0.80      0.85      0.83     25033
    positive       0.84      0.79      0.81     25033

    accuracy                           0.82     50066
   macro avg       0.82      0.82      0.82     50066
weighted avg       0.82      0.82      0.82     50066

